# Redes neuronales: TabNet

## 1.- Descripción

TabNet es un modelo basado en redes neuronales desarrollado específicamente para trabajar con datos tabulares. Combina la capacidad de aprendizaje profundo con la flexibilidad de los árboles de decisión, lo que le permite manejar eficientemente tanto relaciones lineales como no lineales. El modelo se distingue por su arquitectura única que utiliza atención dirigida (decision-aware attention) para seleccionar las características más relevantes durante el entrenamiento, mejorando así la interpretabilidad y el rendimiento.

En este proyecto, TabNet se utiliza para predecir si un préstamo será aceptado o rechazado por un banco, basándose en un conjunto de características del solicitante, como su historial financiero, información personal y otros factores relevantes. Este tipo de problema se ajusta bien al enfoque de TabNet, ya que maneja datos tabulares y es capaz de identificar patrones complejos en los datos mientras mantiene la capacidad de interpretabilidad.

## 2.- Implementación


In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
import torch.optim as optim

In [50]:
# Cargar los datos
df = pd.read_csv(r"C:\Users\Day\Downloads\LBBYs-main\LBBYs-main\data\processed\df_train.csv")
df_test= pd.read_csv(r"C:\Users\Day\Downloads\LBBYs-main\LBBYs-main\data\raw\test_nolabel.csv")

In [52]:
df_test_features = df_test.drop(columns=['id'], errors='ignore') 
df_test_features = df_test.copy()

In [54]:
# Balancear el dataset con undersampling
df_majority = df[df["Accept"] == 1]
df_minority = df[df["Accept"] == 0]
df_majority_downsampled = resample(df_majority, 
                                   replace=False,
                                   n_samples=len(df_minority),
                                   random_state=42)

df_balanced = pd.concat([df_majority_downsampled, df_minority])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


In [56]:
# Separar X e y
X = df_balanced.drop('Accept', axis=1)
y = df_balanced['Accept']

In [64]:
train_columns = X.columns

In [66]:
# Codificar texto si hay columnas categóricas
le = LabelEncoder()
categorical_columns = X.select_dtypes(include=['object']).columns
for col in categorical_columns:
    X[col] = le.fit_transform(X[col].astype(str))

In [68]:
categorical_columns_test = df_test.select_dtypes(include=['object']).columns

categorical_columns_test = [col for col in categorical_columns_test if col != 'id']
for col in categorical_columns_test:
    if col in categorical_columns:  # Solo transformar las columnas que están en df_train
        # Reemplaza las categorías desconocidas con 'Unknown' si es necesario
        if 'Unknown' not in le.classes_:
            le.classes_ = np.append(le.classes_, 'Unknown')
        
        # Reemplaza las categorías desconocidas con 'Unknown' y aplica la transformación
        df_test[col] = df_test[col].astype(str).apply(lambda x: x if x in le.classes_ else 'Unknown')
        df_test[col] = le.transform(df_test[col])

In [72]:
categorical_columns_test = df_test_features.select_dtypes(include=['object']).columns
le = LabelEncoder()

for col in categorical_columns_test:
    df_test_features[col] = le.fit_transform(df_test_features[col].astype(str))


In [74]:
# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
df_test_imputed = imputer.fit_transform(df_test_features)

In [76]:
# Escalar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
df_test_scaled = scaler.fit_transform(df_test_imputed)

In [78]:
# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## 3.- Evaluación y optimización


In [81]:
# Definir el modelo TabNet
tabnet = TabNetClassifier(optimizer_fn=optim.Adam, optimizer_params={'lr': 1e-4})


C:\Users\Day\anaconda3\envs\myenv\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [83]:
# Entrenar el modelo
tabnet.fit(X_train=X_train, y_train=y_train, max_epochs=200, patience=30, batch_size=256, virtual_batch_size=128)

C:\Users\Day\anaconda3\envs\myenv\lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.71489 |  0:00:00s
epoch 1  | loss: 0.7159  |  0:00:01s
epoch 2  | loss: 0.7136  |  0:00:01s
epoch 3  | loss: 0.70938 |  0:00:02s
epoch 4  | loss: 0.70605 |  0:00:03s
epoch 5  | loss: 0.70788 |  0:00:03s
epoch 6  | loss: 0.70399 |  0:00:04s
epoch 7  | loss: 0.70418 |  0:00:04s
epoch 8  | loss: 0.70154 |  0:00:05s
epoch 9  | loss: 0.70305 |  0:00:06s
epoch 10 | loss: 0.70171 |  0:00:06s
epoch 11 | loss: 0.70091 |  0:00:07s
epoch 12 | loss: 0.69985 |  0:00:07s
epoch 13 | loss: 0.69634 |  0:00:08s
epoch 14 | loss: 0.69641 |  0:00:09s
epoch 15 | loss: 0.6959  |  0:00:09s
epoch 16 | loss: 0.6929  |  0:00:10s
epoch 17 | loss: 0.69161 |  0:00:11s
epoch 18 | loss: 0.69312 |  0:00:12s
epoch 19 | loss: 0.6925  |  0:00:12s
epoch 20 | loss: 0.69262 |  0:00:13s
epoch 21 | loss: 0.69126 |  0:00:14s
epoch 22 | loss: 0.68931 |  0:00:15s
epoch 23 | loss: 0.68848 |  0:00:15s
epoch 24 | loss: 0.68808 |  0:00:16s
epoch 25 | loss: 0.68682 |  0:00:17s
epoch 26 | loss: 0.68783 |  0:00:17s
e

In [84]:
# Predecir
y_pred = tabnet.predict(X_test)

In [87]:
y_pred_prob = tabnet.predict_proba(X_test)
y_pred = (y_pred_prob[:, 1] > 0.4).astype(int)  # Umbral de 0.4


In [91]:
# Hacer predicciones
y_test_pred = tabnet.predict(df_test_scaled)

In [93]:
# Evaluar
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[370 373]
 [201 589]]
              precision    recall  f1-score   support

           0       0.65      0.50      0.56       743
           1       0.61      0.75      0.67       790

    accuracy                           0.63      1533
   macro avg       0.63      0.62      0.62      1533
weighted avg       0.63      0.63      0.62      1533



In [95]:
df_test['ACCEPT'] = y_test_pred
print(df_test[['id', 'ACCEPT']].head())
missing_accept = df_test['ACCEPT'].isnull().sum()
print(missing_accept)


            id  ACCEPT
0  6b7ce9ec161       1
1  96123015731       0
2  b2c5181ac5b       1
3  e6cb54a9e6a       1
4  8eddf83466e       1
0


In [97]:
# Crear DataFrame con las predicciones (id y Accept)
df_result = df_test[['id']].copy()  # Agregar 'id' del conjunto de test
df_result['ACCEPT'] = y_test_pred
# Verifica las primeras filas del DataFrame
print(df_result.head())

            id  ACCEPT
0  6b7ce9ec161       1
1  96123015731       0
2  b2c5181ac5b       1
3  e6cb54a9e6a       1
4  8eddf83466e       1


In [99]:
# Guardar el resultado en un archivo CSV
df_result.to_csv("SUBMMIT_TABNET.csv", index=False)

print("¡Predicciones completas y archivo guardado como 'resultado_test.csv'!")

¡Predicciones completas y archivo guardado como 'resultado_test.csv'!


## 4.- Conclusión

En el modelo de predicción de aceptación de préstamos utilizando TabNet, se observó que los resultados obtenidos presentan una precisión moderada, con un valor de accuracy de 0.63. El modelo muestra un buen desempeño en la clase minoritaria (préstamos aceptados), alcanzando un recall de 0.75, lo que indica que es bastante efectivo para identificar los casos positivos (préstamos aceptados). Sin embargo, su desempeño en la clase mayoritaria (préstamos rechazados) es más bajo, con un recall de 0.50, lo que sugiere que hay una tasa considerable de falsos negativos en esta clase.

Aunque el f1-score para la clase 1 (préstamos aceptados) es razonable (0.67), el f1-score para la clase 0 (préstamos rechazados) es más bajo (0.56), lo que refleja un desequilibrio en el rendimiento entre las dos clases. En términos generales, el modelo podría beneficiarse de ajustes adicionales para mejorar la identificación de los préstamos rechazados y equilibrar mejor la predicción de ambas clases.

Por lo tanto, aunque TabNet tiene una capacidad destacada para manejar datos tabulares, su rendimiento en este caso no supera a otros modelos, lo que sugiere que podría ser necesario optimizar parámetros adicionales o emplear técnicas de balanceo de clases más efectivas.